In [1]:
!pip install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np
from eep153_tools.sheets import read_sheets
Ethiopia_Data = '1qOZi1FqAS89x461pCIgwRaAfpBw5aUp7kBbp0tQKI_E'
# Change 'Ethiopia_Data' to key of your own sheet in Sheets, above
x = read_sheets(Ethiopia_Data,sheet = 'Food Expenditures (2011-12)')
x = x.set_index(['i','t','m','j']).squeeze()


# Now prices
p = read_sheets(Ethiopia_Data,sheet='Food Prices (2011-12)').set_index(['t','m','j','u'])

# Compute medians of prices for particular time, place and unit
p = p.groupby(['t','m','j','u']).median()

# Just keep metric units
p = p.xs('Kg',level="u").squeeze().unstack('j')

# Drop prices for goods we don't have expenditures for
# p = p[x.index.levels[-1]].T
p =  p.dropna()

# Household characteristics
d = read_sheets(Ethiopia_Data,sheet="Household Characteristics")
d.columns.name = 'k'

# Fill blanks with zeros
d = d.replace(np.nan,0)

# Expenditures x may have duplicate columns
x = x.T.groupby(['i','t','m','j']).sum()
x = x.replace(0,np.nan) # Replace zeros with missing

# Take logs of expenditures; call this y
y = np.log(x)

d.set_index(['i','t','m'],inplace=True)

In [2]:
d.groupby('t').mean()

k,Females 00-03,Females 04-08,Females 09-13,Females 14-18,Females 19-30,Females 31-50,Females 51-99,Males 00-03,Males 04-08,Males 09-13,Males 14-18,Males 19-30,Males 31-50,Males 51-99,log HSize
t,,,,,,,,,,,,,,,
2011-12,0.000000,0.332397,0.341843,0.272913,0.470258,0.424560,0.201174,0.000000,0.341843,0.368394,0.295124,0.406178,0.378095,0.246362,1.240192
2013-14,0.234645,0.345883,0.349306,0.319643,0.549724,0.436965,0.226849,0.249097,0.357102,0.354440,0.321354,0.467579,0.404640,0.229131,1.407929
2015-16,0.217172,0.340404,0.372727,0.289899,0.498788,0.487677,0.254747,0.240000,0.356364,0.375152,0.312929,0.420000,0.425051,0.207879,1.420437
2018-19,0.216248,0.320384,0.286263,0.280798,0.536780,0.428508,0.173855,0.224963,0.316987,0.293205,0.266470,0.424225,0.402659,0.186706,1.302566


In [5]:
pip install cfe

  Using cached cfe-0.0.22-py3-none-any.whl.metadata (649 bytes)
Using cached cfe-0.0.22-py3-none-any.whl (2.1 kB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install cfe==0.6.4.dev0

ERROR: Could not find a version that satisfies the requirement cfe==0.6.4.dev0 (from versions: 0.0.22)
ERROR: No matching distribution found for cfe==0.6.4.dev0
Note: you may need to restart the kernel to use updated packages.


In [17]:
import cfe.regression as rgsn

result = rgsn.Regression(y=y, d=d)

AssertionError: Fewer rows than columns.  Consider passing the transpose.

In [19]:
y.unstack('j').count(axis=1,min_prop_items=0.01)

TypeError: DataFrame.count() got an unexpected keyword argument 'min_prop_items'

In [16]:
y.groupby('j').count()

j
Banana            361
Barley            170
Chat/Kat          432
Cheese             48
Chick Pea         169
Coffee           2040
Eggs              217
Field Pea         655
Haricot Beans     272
Horsebeans        918
Kocho             231
Lentils           504
Linseed           172
Maize             726
Meat              905
Milk              262
Millet             79
Niger Seed         28
Onion            2572
Potato            905
Salt             3670
Sorghum           414
Sugar            1877
Teff              643
Wheat             635
Name: Expenditure, dtype: int64

In [5]:
import cfe
cfe.__version__

AttributeError: module 'cfe' has no attribute '__version__'

In [4]:
InputFiles = {'Expenditures':('1qOZi1FqAS89x461pCIgwRaAfpBw5aUp7kBbp0tQKI_E','Expenditures'),
              'Household Characteristics':('1qOZi1FqAS89x461pCIgwRaAfpBw5aUp7kBbp0tQKI_E','Household Characteristics'),
              'Food Prices':('1qOZi1FqAS89x461pCIgwRaAfpBw5aUp7kBbp0tQKI_E','Food Prices (per KG ETB)')}

In [7]:
from eep153_tools.sheets import read_sheets
import numpy as np
import pandas as pd

def get_clean_sheet(key,sheet=None):

    dfs = read_sheets(key)

    if sheet is not None:
        df = dfs[sheet]
    else:
        df = dfs.values()[0]
        
    df.columns = [c.strip() for c in df.columns.tolist()]

    df = df.loc[:,~df.columns.duplicated(keep='first')]   

    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)

    df = df.loc[~df.index.duplicated(), :]

    return df

ModuleNotFoundError: No module named 'gspread.models'

In [6]:
# Get expenditures...
x = get_clean_sheet(InputFiles['Expenditures'][0],
                    sheet=InputFiles['Expenditures'][1])

if 'm' not in x.columns:
    x['m'] = 1

x = x.set_index(['j','t','m'])
x.columns.name = 'i'

x = x.apply(lambda x: pd.to_numeric(x,errors='coerce'))
x = x.replace(0,np.nan)

x

NameError: name 'read_sheets' is not defined

In [8]:
# Get Household characteristics...
z = get_clean_sheet(InputFiles['Household Characteristics'][0],
                    sheet=InputFiles['Household Characteristics'][1])

if 'm' not in z.columns:
    z['m'] = 1

z = z.set_index(['j','t','m'])
z.columns.name = 'k'

z = z.apply(lambda x: pd.to_numeric(x,errors='coerce'))

z

NameError: name 'read_sheets' is not defined

In [ ]:
q = get_clean_sheet(InputFiles['Food Prices'][0],
                    sheet=InputFiles['Food Prices'][1])

if 'm' not in q.columns:
    q['m'] = 1

q = q.set_index(['j','t','m','u'])
q.columns.name = 'i'

q = q.apply(lambda x: pd.to_numeric(x,errors='coerce'))
q = q.replace(0,np.nan)

q